In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

# Đọc dữ liệu từ tệp CSV với tiêu đề cột
data = pd.read_csv("image_data.csv")

# Tách nhãn và đặc trưng
y = data.iloc[:, 0].values  # Nhãn là cột đầu tiên
X = data.iloc[:, 1:].values  # Đặc trưng là các cột còn lại

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Xây dựng mô hình
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')  # Số units cuối cùng phải bằng số lớp
])

# Biên soạn mô hình
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

# Lưu mô hình
model.save("MODEL/hand_model")

# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):
    c_str = ''

    # Create header guard
    c_str += '#ifndef ' + var_name.upper() + '_H\n'
    c_str += '#define ' + var_name.upper() + '_H\n\n'

    # Add array length at top of file
    c_str += 'const unsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

    # Declare C variable
    c_str += 'const unsigned char ' + var_name + '[] = {'
    hex_array = []
    for i, val in enumerate(hex_data):
        # Construct string from hex
        hex_str = format(val, '#04x')

        # Add formatting so each line stays within 80 characters
        if (i + 1) < len(hex_data):
            hex_str += ','
        if (i + 1) % 12 == 0:
            hex_str += '\n '
        hex_array.append(hex_str)

    # Add closing brace
    c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

    # Close out header guard
    c_str += '#endif //' + var_name.upper() + '_H'

    return c_str

tflite_model_name = 'MODEL/hand_model'  # Will be given .tflite suffix
c_model_name = 'MODEL/hand_model'       # Will be given .h suffix

# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

# Write TFLite model to a C source (or header) file
with open(c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model, c_model_name))


Epoch 1/100
1/1 [==============================] - 0s 418ms/step - loss: 79.9182 - accuracy: 0.5000 - val_loss: 128.4196 - val_accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 33ms/step - loss: 129.9234 - accuracy: 0.5000 - val_loss: 56.3777 - val_accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 21ms/step - loss: 62.7179 - accuracy: 0.5000 - val_loss: 17.3887 - val_accuracy: 0.5000
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 10.4377 - accuracy: 0.5000 - val_loss: 11.7208 - val_accuracy: 0.7500
Epoch 5/100
1/1 [==============================] - 0s 21ms/step - loss: 6.4770 - accuracy: 0.6250 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 20ms/step - loss: 5.0394 - accuracy: 0.9375 - val_loss: 2.2194 - val_accuracy: 0.7500
Epoch 7/100
1/1 [==============================] - 0s 16ms/step - loss: 10.8062 - accuracy: 0.6875 - val_loss: 3.2783e-07 - val_accuracy: 1.0000


INFO:tensorflow:Assets written to: MODEL/hand_model\assets


INFO:tensorflow:Assets written to: C:\Users\tranh\AppData\Local\Temp\tmpg0uhzfhw\assets


INFO:tensorflow:Assets written to: C:\Users\tranh\AppData\Local\Temp\tmpg0uhzfhw\assets
